In [1]:
!pip install flask_cors

  Obtaining dependency information for flask_cors from https://files.pythonhosted.org/packages/56/07/1afa0514c876282bebc1c9aee83c6bb98fe6415cf57b88d9b06e7e29bf9c/Flask_Cors-5.0.0-py2.py3-none-any.whl.metadata


In [35]:
from flask import Flask, jsonify, request
from rdflib import Graph, Namespace, URIRef,Literal
import geopandas as gpd
from rdflib.namespace import RDF, RDFS
import json
from rdflib import Graph, Namespace, RDF, RDFS
from shapely import wkt
import geopandas as gpd
from shapely.ops import transform
import pyproj

In [3]:
app = Flask(__name__)

# Load the ontology
g = Graph()
g.parse('.backend/rdf_data/tamu_ont.owl')

# Define namespace
GEO = Namespace('http://www.opengis.net/ont/geosparql#')
TAMU_ONTO = Namespace('http://tamu.edu/ontologies/tamu_ont#')

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#integer, Converter=<class 'int'>
Traceback (most recent call last):
  File "/Users/lihang/anaconda3/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: 'None'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#integer, Converter=<class 'int'>
Traceback (most recent call last):
  File "/Users/lihang/anaconda3/lib/python3.11/site-packages/rdflib/term.py", line 2119, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: 'None'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#integer, Converter=<class 'int'>
Traceback (most recent call last):
  File 

In [23]:
project_to_wgs84 = pyproj.Transformer.from_crs('epsg:3857', 'epsg:4326', always_xy=True).transform

def extract_property(subject, property):
    obj = g.value(subject, property)
    return str(obj) if obj else None

def convert_geometry(wkt_str):
    try:
        geom = wkt.loads(wkt_str)  # 将 WKT 字符串转换为 shapely 几何对象
        geom_transformed = transform(project_to_wgs84, geom)  # 转换坐标系
        return geom_transformed.wkt  # 返回转换后的 WKT 字符串
    except Exception as e:
        print(f"Error converting geometry: {e}")
        return None

buildings = []

# 假设 g 是预先加载的 RDF 图，并且包含建筑物数据
for s in g.subjects(RDF.type, TAMU_ONTO.Building):
    geometry_wkt = extract_property(s, GEO.hasGeometry)
    geometry_converted = convert_geometry(geometry_wkt) if geometry_wkt else None
    
    building_data = {
        'id': str(s),
        'name': extract_property(s, RDFS.label),
        'address': extract_property(s, TAMU_ONTO.hasAddress),
        'yearBuilt': extract_property(s, TAMU_ONTO.yearBuilt),
        'numFloors': extract_property(s, TAMU_ONTO.numFloors),
        'geometry': geometry_converted  # 转换为 WGS 1984 坐标系后的 WKT 格式
    }
    buildings.append(building_data)

# 使用 json.dumps 来在 Jupyter Notebook 中查看数据
j = json.dumps({'buildings': buildings}, indent=4)
print(j)

{
    "buildings": [
        {
            "id": "http://tamu.edu/ontologies/tamu_ont#University_Building_904",
            "name": "Becky Gates Childrens Center Multipurpose Bldg",
            "address": "1121 Hensel Dr",
            "yearBuilt": "2009",
            "numFloors": "1",
            "geometry": "POLYGON ((-96.34310904007785 30.62832546116682, -96.34318383294855 30.62820024408481, -96.34297199965367 30.62810561805736, -96.34289720658084 30.62823083501668, -96.34310904007785 30.62832546116682))"
        },
        {
            "id": "http://tamu.edu/ontologies/tamu_ont#Non_University_Building_154",
            "name": "Executive Office Plaza, Bryan Tx (VACATED)",
            "address": "3608 W 29th St",
            "yearBuilt": "2004",
            "numFloors": "2",
            "geometry": "POLYGON ((-96.33536526396178 30.64998828931511, -96.33532371227449 30.64987348518912, -96.3353100594271 30.64987716797467, -96.33530394298607 30.64986030527255, -96.33509428621795 30.649

In [29]:
parking_lots = []

for s in g.subjects(RDF.type, TAMU_ONTO.ParkingLot):
    geometry_wkt = extract_property(s, GEO.hasGeometry)
    geometry_converted = convert_geometry(geometry_wkt) if geometry_wkt else None
    
    parking_lot_data = {
        'id': str(s),
        'name': extract_property(s, RDFS.label),
        'lotType': extract_property(s, TAMU_ONTO.lotType),
        'area': extract_property(s, TAMU_ONTO.area),
        'length': extract_property(s, TAMU_ONTO.length),
        'geometry': geometry_converted  # 转换为 WGS 1984 坐标系后的 WKT 格式
    }
    parking_lots.append(parking_lot_data)

# 使用 json.dumps 来在 Jupyter Notebook 中查看数据
jp = json.dumps({'parking_lots': parking_lots[:5]}, indent=4)
print(jp)

{
    "parking_lots": [
        {
            "id": "http://tamu.edu/ontologies/tamu_ont#ParkingLot_26288",
            "name": "63",
            "lotType": "Surface",
            "area": "89352.44734129",
            "length": "4413.87484835",
            "geometry": "POLYGON ((-96.34357992571779 30.59899098935146, -96.34369710504316 30.59888834366047, -96.34371954792311 30.5988726424217, -96.34374425219684 30.59886081403966, -96.34377055555107 30.59885317562921, -96.34379775280205 30.5988499319735, -96.34382511480057 30.59885117003384, -96.34385190798106 30.59885685661828, -96.3438774140273 30.59886683927141, -96.343900949131 30.59888085036171, -96.3436096156027 30.59863246443044, -96.34362434467099 30.59864974240901, -96.34363597800412 30.59866923957689, -96.34364418817383 30.59869040717265, -96.34364874409889 30.59871264941942, -96.34364951754988 30.59873534029355, -96.34364648675735 30.59875784114447, -96.34363973702509 30.59877951866993, -96.34362945832903 30.59879976274076, -96.

In [81]:
SOSA = Namespace('http://www.w3.org/ns/sosa/')
def get_occupancy(parking_instance):
    # 查找与停车场相关的 Observation
    for observation in g.subjects(SOSA.hasFeatureOfInterest, parking_instance):
        # 获取占用率值
        occupancy_value = extract_property(observation, SOSA.hasSimpleResult)
        if occupancy_value:
            return occupancy_value
    return None

def get_nearby_parking(building_name):
    # 去除输入名称的空格并转为小写
    building_name = building_name.strip().lower()
    building_uri = None

    # 查找匹配的建筑物
    for s, label_literal in g.subject_objects(RDFS.label):
        label = str(label_literal).strip().lower()
        if label == building_name and (s, RDF.type, TAMU_ONTO.Building) in g:
            building_uri = s
            break

    if not building_uri:
        return {'error': 'Building not found'}

    # 查询关联的 nearby 停车场
    nearby_parking = []
    for o in g.objects(building_uri, TAMU_ONTO.nearby):
        if (o, RDF.type, TAMU_ONTO.ParkingLot) in g:
            geometry_wkt = extract_property(o, GEO.hasGeometry)
            geometry_converted = convert_geometry(geometry_wkt) if geometry_wkt else None
            
            # 获取停车场的占用率
            occupancy_value = get_occupancy(o)
            
            parking_lot_data = {
                'id': str(o),
                'name': extract_property(o, RDFS.label),
                'lotType': extract_property(o, TAMU_ONTO.lotType),
                'area': extract_property(o, TAMU_ONTO.area),
                'length': extract_property(o, TAMU_ONTO.length),
                'geometry': geometry_converted,
                'occupancy': occupancy_value
            }
            nearby_parking.append(parking_lot_data)

    return {'nearby_parking': nearby_parking}

# 测试 get_nearby_parking
building_name = "Century Square #09"  # 替换为要查询的建筑名称
nearby_parking_data = get_nearby_parking(building_name)

# 将结果转换为 JSON 格式并打印
k = json.dumps(nearby_parking_data, indent=4)
print(k)

{
    "nearby_parking": [
        {
            "id": "http://tamu.edu/ontologies/tamu_ont#ParkingLot_26464",
            "name": "50",
            "lotType": "Surface",
            "area": "173178.97704413",
            "length": "4264.99772016",
            "geometry": "POLYGON ((-96.33689548731006 30.62487786266804, -96.33689925430104 30.6248757517693, -96.33690332934745 30.62487432344437, -96.33690758988706 30.6248736206519, -96.3369119077788 30.62487366452925, -96.33691615315684 30.62487445375672, -96.33692019833613 30.62487596459734, -96.33692392165283 30.62487815161072, -96.33692741130876 30.62488104572843, -96.33693134145231 30.62488330572566, -96.33693559809342 30.62488486605333, -96.33694005777227 30.62488568145563, -96.33707876392734 30.62489622300935, -96.33710406398494 30.62489499998909, -96.33711195639214 30.62489375998576, -96.33711957151033 30.62489134385788, -96.33712673513185 30.6248878068781, -96.33713328337777 30.62488322996039, -96.33782364667778 30.62425705660372,

In [75]:
building_uri

rdflib.term.URIRef('http://tamu.edu/ontologies/tamu_ont#University_Building_192')